In [1]:
import keras
from keras import layers
import gymnasium as gym
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing
import numpy as np
import tensorflow as tf
import ale_py


In [2]:
import logging
from gymnasium.wrappers import RecordEpisodeStatistics
import json

In [3]:

gym.register_envs(ale_py)


In [4]:
env = gym.make('SpaceInvadersNoFrameskip-v4', render_mode="rgb_array")

In [5]:

gamma = 0.99 # discount_factor
epsilon = 0.5
epsilon_min = 0.1
epsilon_max = 0.5
epsilon_interval = (epsilon_max - epsilon_min)
batch_size = 32
max_steps_per_episode = 10000 
max_episodes = 0 
max_frames = 1e7 

env = AtariPreprocessing(env)

env = FrameStack(env, 4)
trigger = lambda t: t % 50 == 0
env = gym.wrappers.RecordVideo(env, video_folder="./Outputs/videos", name_prefix='training', episode_trigger=trigger, disable_logger=False)
env = RecordEpisodeStatistics(env)

num_actions = 4

d:\workdir\DeepQLab\.venv\Lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at d:\workdir\DeepQLab\Outputs\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [6]:
model = keras.models.load_model("breakout_qmodel_7764.keras", safe_mode = False)


In [7]:

saved_model = keras.models.load_model("breakout_qmodel_7764.keras", safe_mode = False)


In [46]:
def create_q_model():
    return keras.Sequential(
        [
            layers.Lambda
            (
                lambda tensor: keras.ops.transpose(tensor, [0, 2, 3, 1]),
                output_shape=(84, 84, 4),
                input_shape=(4, 84, 84)
            ),
            layers.Conv2D(32, kernel_size=8, strides=4, activation="leaky_relu"),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="leaky_relu"),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="leaky_relu"),
            layers.Flatten(),
            layers.Dense(512, activation="leaky_relu"),
            layers.Dense(num_actions, activation="linear")
        ]
    )
model = create_q_model()
model_target = create_q_model()
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)


action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 7764
frame_count = 4930000

# Number of frames to take random action and observe output
epsilon_random_frames = 10000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0 - frame_count
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 10000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()

for layer in model.layers:
    if layer.name in [l.name for l in saved_model.layers]:
        layer.set_weights(saved_model.get_layer(layer.name).get_weights())

d:\workdir\DeepQLab\.venv\Lib\site-packages\keras\src\layers\core\lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [47]:
for layer in model_target.layers:
    if layer.name in [l.name for l in saved_model.layers]:
        layer.set_weights(saved_model.get_layer(layer.name).get_weights())

In [48]:
statistics = []

In [ ]:
while True:
    observation, info = env.reset()
    state = np.array(observation)
    episode_reward = 0


    for timestep in range(1, max_steps_per_episode):
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            try:
            # Predict action Q-values
            # From environment state
                state_tensor = keras.ops.convert_to_tensor(state)
                state_tensor = keras.ops.expand_dims(state_tensor, 0)
                action_probs = model(state_tensor, training=False) 
                # Take best action
                action = keras.ops.argmax(action_probs[0]).numpy()
            except:
                raise Exception(f"State tensor shape {state_tensor.shape}, model input shape: {model.input_shape}")

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _, info = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # Get indices of samples for replay buffers
            indices = np.random.choice(
                range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array(
                [state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample, verbose=0)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * keras.ops.amax(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * \
                (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = keras.ops.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = keras.ops.sum(
                    keras.ops.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            print(f"best score of last 100: {np.max(episode_reward_history)}, running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}")
            stat_dic = {'best_score': np.max(episode_reward_history), 'running_reward': running_reward, 
                        'episode': episode_count, 'frame_count': frame_count}
            statistics.append(stat_dic)
            with open(f"./Outputs/statistics_{episode_count}.json", "w") as json_file:
                json.dump(statistics, json_file)
            model.save(f"./Outputs/breakout_qmodel_{episode_count}.keras")

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    logging.info(f"Episode {episode_count}, info['episode']")

    episode_count += 1

    if running_reward > 1000:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

    if (
        max_episodes > 0 and episode_count >= max_episodes
    ):  # Maximum number of episodes reached
        print("Stopped at episode {}!".format(episode_count))
        break
    if (max_frames <= frame_count):
        print(f"Stopped at frame {frame_count}!")

d:\workdir\DeepQLab\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-0.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-0.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-0.mp4
best score of last 100: 295.0, running reward: 134.47 at episode 635, frame count 350000
best score of last 100: 415.0, running reward: 148.78 at episode 653, frame count 360000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-50.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-50.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-50.mp4
best score of last 100: 415.0, running reward: 150.09 at episode 671, frame count 370000
best score of last 100: 425.0, running reward: 148.78 at episode 690, frame count 380000
best score of last 100: 425.0, running reward: 153.57 at episode 707, frame count 390000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-100.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-100.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-100.mp4
best score of last 100: 425.0, running reward: 153.20 at episode 724, frame count 400000
best score of last 100: 425.0, running reward: 151.10 at episode 745, frame count 410000
best score of last 100: 440.0, running reward: 154.80 at episode 762, frame count 420000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-150.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-150.mp4



...

MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-6950.mp4
best score of last 100: 525.0, running reward: 214.65 at episode 7577, frame count 4800000
best score of last 100: 525.0, running reward: 219.30 at episode 7591, frame count 4810000
best score of last 100: 525.0, running reward: 210.25 at episode 7608, frame count 4820000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-7000.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-7000.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-7000.mp4
best score of last 100: 525.0, running reward: 207.90 at episode 7624, frame count 4830000
best score of last 100: 525.0, running reward: 202.80 at episode 7641, frame count 4840000
best score of last 100: 565.0, running reward: 213.85 at episode 7653, frame count 4850000
best score of last 100: 565.0, running reward: 227.05 at episode 7666, frame count 4860000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-7050.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-7050.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-7050.mp4
best score of last 100: 565.0, running reward: 232.25 at episode 7679, frame count 4870000
best score of last 100: 860.0, running reward: 236.05 at episode 7691, frame count 4880000
best score of last 100: 860.0, running reward: 247.85 at episode 7706, frame count 4890000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-7100.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-7100.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-7100.mp4
best score of last 100: 860.0, running reward: 249.15 at episode 7722, frame count 4900000
best score of last 100: 860.0, running reward: 254.65 at episode 7735, frame count 4910000
best score of last 100: 860.0, running reward: 259.60 at episode 7749, frame count 4920000
best score of last 100: 860.0, running reward: 250.05 at episode 7764, frame count 4930000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-7150.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-7150.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-7150.mp4
